# Nerfstudio To D-NeRF dataset

Set Up:

1. Installl Nerfstudio and activate environment
2. Run this notebook with the nerfstudio conda environment

Instructions:

1. Update configuration relative to desired parameters
2. Run the method

---
# Configuration
---

Information:
1. `nerfstudio_fp` is the path **to the folder** containing `transforms.json`
2. `output_fp` is the path to the folder you wish to write `transforms_train.json`, `transforms_test.json` and `transforms_val.json`
3. `downscale_images_fp` is the folder name inside of you nerfstudio folder containing the downscaled images if you would rather save these than the original images. As dnerf format doesn't consider downscaled images, this will allow you to use them instead.
4. `method` declares the way we recover the time values:
    - `'exhaustive'`    : match image to frame (**super slow**)
    - `'linear'`    : assign time given image name-index (**fast**) (e.g. image w/name `frame_{i}.png` is going to be at i/n time where n is the number of images and `i` is the index; this is the image-naming format used by nerfstudio-colamp process)

In [1]:
nerfstudio_fp ='path/to/folder/containing/tranform/file'
video_fp = 'path/to/video.mp4'
output_fp = 'path/to/write/transform/to'

downscale_images_fp = 'path/to/folder/containing/downscalled/images'

method = 'exhaustive'

In [9]:
nerfstudio_fp ='data/boat_colmap/'
video_fp = 'data/boat/boat.mp4'
output_fp = 'data/boat_colmap_/'

downscale_images_fp = 'images/'

method = 'exhaustive'

---
# View the code
---

Information:
1. Import dependencies. *Make sure nerfstudio has been downloaded*
2. View the functions
3. Change the functions (optional)
4. Run the functions

In [15]:
# Import
import os
import json
from pathlib import Path
import random

import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from tqdm import tqdm

from utils_ import *

### Exhaustive Image Search

**Args:**
- d_fp, v_fp, img_fp: Path, previously discussed
- transfors_fp: Path, path to transforms file

**Notes:**
1. Exhaustive search matches each image (e.g. `frame_0000.png`, `frame_0001.png`, ... ) to each frame in a video. 
2. Each image is a frame with png compression so direct image to frame comparison isn't possible
3. Instead we compare w/ SSIM.
4. This means:

    a. Overlapping frames (such as monocular stationary camera with negligible dynamic motion) will have the same SSIM score and so we will get several frames which match
        
    b. We select the earliest occuring frame match as the time of the png image
    
    c. We accept that this may not always be the case so we add a threshold, whereby we search for the earliest match where SSIM = 1.0, when this is not the case max(SSIM) > 0.98 is selected.
    
    d. Theoretically, this shouldn't be an issue for NeRF as SSIM threshold is high
    

In [ ]:
def exhaustive(d_fp, o_fp, v_fp, img_fp, transforms_fp, shuffle:bool=True):
    assert not os.path.exists(o_fp), 'Folder already exists, delete folder to run'

    os.makedirs(o_fp)
    with open(transforms_fp) as fp:
        contents = fp.read()
    transforms = json.loads(contents)
    img_frames = transforms['frames'] # Directly access frame data

    # Initialise opencv video object
    video = cv2.VideoCapture(str(v_fp))
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f'Total number of frames to process {total_frames} \n Total number of images to process {len(img_frames)}')


    video_frame_counter = 0
    matches = []

    # Sort img_frames by filepath name
    filenum = []
    for img in img_frames:
        num = img['file_path'].split('/')[-1].split('.')[-2].split('_')[-1]
        filenum.append(int(num))
    img_frames = [img for _, img in sorted(zip(filenum, img_frames))]
        

    iterator = tqdm(enumerate(img_frames))
    # loop through each image in our colmap dataset
    for idx, img in iterator:
        fp = img['file_path']
        image = cv2.imread(fp, cv2.IMREAD_GRAYSCALE) # load image greyscale

        SSIM = {
            'max': 0.,
            'idxs':[]
        }
        for idx_video in range(video_frame_counter, total_frames):
            # Fetch frame from video
            video.set(cv2.CAP_PROP_FRAME_COUNT, i)
            ret, frame = video.read()
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # load frame greyscale

            # Get SSIM
            ssim_res = ssim(image, frame)

            # Process SSIM
            if ssim_res == 1.0: # if we meet ideal match
                SSIM['max'] = 1.
                SSIM['idxs'] = [idx_video] 
                break
            elif ssim_res > SSIM: # if we have a new max
                SSIM['max'] = ssim_res
                SSIM['idxs'] = [idx_video]
            elif ssim_res == SSIM: # if we have the same max
                SSIM['idxs'].append(idx_video)
        

        if SSIM['max'] < 0.98:
            print(f'Image {idx} has no match: consider lowering threshold')
        else:
            idx_video = min(SSIM['idxs'])
            matches.append({
                "frame" : float(idx_video/total_frames),
                "image" : idx
                })

    # shuffle data
    if shuffle == True:
        random.shuffle(matches)

    # train split
    train_split = 0.9
    train_idx = int(train_split * len(matches))
    train_data = matches[:train_idx]
    # test split (on remaining data)
    test_split = 0.9
    test_index = train_idx + int(test_split * (len(matches) - len(train_data)))
    test_data =  matches[train_idx : test_index]
    # val split
    val_data = matches[test_index:]


    # Construct transform files
    local_properties = {
        "roation": 0.0,
    }
    data = [train_data, test_data, val_data]
    for idx, d in enumerate(data):
        file_ = {
            "camera_angle_x": 0.0,
            "frames":[]
        }
        # file_path
        if idx == 0: file_path = Path(o_fp) / 'transforms_train.json'
        elif idx == 1: file_path = Path(o_fp) / 'transforms_test.json'
        elif idx == 2: file_path = Path(o_fp) / 'transforms_val.json'
        
        for match_idx in d:
            img_data = img_frames[match_idx]
            time = float(img_data['idx'] / total_frames)
            fname = img_data['data']['file_path'].split('/')[-1]
            
            file_["frames"].append({
            "file_path":f'./train/{fname}',
            "rotation": local_properties['rotation'],
            "time":time,
            "transform_matrix":img_data['data']['transform_matrix']
        })




### Handler for nerfstudio2dnerf

**Args:**
- `d_fp`, Path, path to `transforms.json` **folder**
- `o_fp`, Path, path to output folder
- `v_fp`, Path, path to video
- `img_fp`, Path, path to image folder

In [ ]:
def handler(d_fp, o_fp, v_fp, img_fp, meth):
    d_fp = Path(d_fp)
    v_fp = Path(v_fp)
    img_fp = d_fp / img_fp

    transforms_fp = d_fp/'transforms.json'

    # Sanity Checks
    pathchecks([d_fp, v_fp, img_fp])
    folderchecks([d_fp, img_fp])
    
    # Handle exhaustive method
    if meth == 'exhaustive':
        exhaustive(d_fp, o_fp, v_fp, img_fp,transforms_fp)    

handler(nerfstudio_fp, output_fp, video_fp, downscale_images_fp, method)


FileExistsError: [Errno 17] File exists: 'data/boat_colmap_/'

---
# Run the code
---